In [7]:
import requests
import random
import json
import base64
from IPython.display import display, SVG

In [8]:
registryUrl = "http://localhost:8081"
keycloak_url = "http://localhost:8080"
bulkIssuanceUrl = "http://localhost:5665"

In [18]:
#create issuer token
issuerUserId = 'scholarship'
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = {
  'client_id': 'registry-frontend',
  'username': issuerUserId,
  'password': 'abcd@123',
  'grant_type': 'password'
}

response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloak_url), headers=headers, data=data)
issuerToken = response.json()["access_token"]
print(response.status_code)
print(issuerToken)

200
eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZUHc4MWFDNjNRUUpYSnAyWVhkbTh4amxxaDZaZ05BRVp5MXlEdlBTTVQwIn0.eyJleHAiOjE2ODE5NzAxMTcsImlhdCI6MTY4MTk2OTUxNywianRpIjoiMTczZDViNzMtYzg2Mi00YmNhLWEyZTktOTVhY2Y0NGVmOTI2IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIyNjVlY2EyNi1kZTA4LTRlODUtYTM2Mi1jMTU4NzRiNDM2NDkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJjZjlmMzdjZi0wYmFlLTQ4ZjQtOTk4ZS0xYjAyMWE4ZTZiYzIiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIklzc3VlciIsInNjaG9sYXJzaGlwIiwib2ZmbGluZV9hY2Nlc3MiLCJhZG1pbiIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1uZGVhciJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5h

In [22]:
schemaName = 'PreMatricScholarship'
headers = {
    'Authorization': 'Bearer %s'%(issuerToken)
}

In [27]:
# get sample csv file for a schema
response = requests.get('%s/v1/bulk/sample/%s'%(bulkIssuanceUrl, schemaName), headers=headers)
print(response.content)

b'validupto,nameofScheme,name,email,contact,institute,academicYear,gender,rollNo,dateofaward\n2023-04-20,string,string,string,string,string,string,string,string,2023-04-20\n'


In [19]:
# upload a csv file to create records
files=[
  ('file',('sample.csv',open('/Users/tejasvarade/Documents/Sahaj/sunbird-rc/demo-education-registry/schema_templates/sample.csv','rb'),'text/csv'))
]
response = requests.post('%s/v1/uploadFiles/%s'%(bulkIssuanceUrl, schemaName), headers = headers, files=files)
print(response.status_code)
print(response.json())

200
{'error': 0, 'success': 1, 'totalRows': 1}


In [21]:
#get all uploaded files
response = requests.get('%s/v1/bulk/uploadedFiles'%(bulkIssuanceUrl), headers=headers)
uploadedFileId = response.json()[-1]['ID']
print(uploadedFileId)

1


In [28]:
# download a report file
response = requests.get('%s/v1/download/%s'%(bulkIssuanceUrl, uploadedFileId), headers=headers)
print(response.content)

b'nameofScheme,gender,academicYear,rollNo,validupto,dateofaward,name,email,contact,institute\nPreMatricScholarship,Male,2023-2024,51,2024-04-20,2023-04-20,Sharad,sharad@gmail.com,7788996655,Institute5\n'
